#  Track Analysis

In order to properly analyze our GPX tracks, we need to put all of them into the same _frame-of-reference_.   This is accomplished by the following algorithm...

* Create a set of variables with the initialized values
    * Current Coordinate = Starting Coordinate
        * $P_{\textrm{cur}} = P_{\textrm{start}}$
        * **NOTE:** Starting coordinate $P_{\textrm{start}}$ is defined below as a fixed variable
    * Distance Threshold = Small distance in meters
        * $d_{\textrm{thresh}} = 20 meters$
        * This is the distance between a possible point and the current point for consideration.
    * Step Distance = Small distance in meters.
        * $d_{\textrm{step}} = 20 meters$
        * This is the distance that the route will jump using the average angle
    * Waypoint list = empty array
        * $\hat{P}_{\textrm{waypoints}} = []$

## Step 0: Globals

In [1]:
database_path = 'bike_data.db'

start_coord = (39.5989743, -104.8609468)
end_coord   = (39.75428108249532, -105.00085402872664)

dist_thresh_m = 20
step_dist_m = 25

## Step 1: Import Required Libraries

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from ipyleaflet import Map, Marker, Polygon, Polyline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import math, cmath
from geographiclib.geodesic import Geodesic

Setup the database.

In [3]:
# SQLAlchemy connectable 
conn = create_engine( 'sqlite:///' + database_path ).connect()

#  For each segment, we need to create a track for each dataset
dataset_ids = pd.read_sql_query('SELECT DISTINCT datasetId FROM point_list', conn)

## Step 2: Import points 

In [4]:
#  For each dataset, load the points w.r.t. each dataset-id, sorted by time.
points_by_dataset = {}
for dataset_id in dataset_ids['datasetId']:
        
    #  Create a full track of the segment
    sql_query = 'SELECT * FROM point_list WHERE datasetId = {} ORDER BY timestamp'.format( dataset_id )
    points_by_dataset[dataset_id] = { 'points': pd.read_sql_query( sql_query, conn ) }

## Step 3: Compute Initial Starting Point

In [5]:
def mean_angle(angles):
    return math.degrees(cmath.phase(sum(cmath.rect(1, math.radians(d)) for d in angles)/len(angles)))

waypoint_list = []
current_point = start_coord
matching_points = []

current_dataset_idx = {}
for dataset_id in dataset_ids['datasetId']:
    current_dataset_idx[dataset_id] = 0

#  Start a loop, running until all points are complete
while True:
    
    #  Look for the nearest first points within the threshold
    temp_angles = []
    for dataset_id in dataset_ids['datasetId']:
        current_point = ( points_by_dataset[dataset_id]['points']['latitude'][current_dataset_idx[dataset_id]],
                          points_by_dataset[dataset_id]['points']['longitude'][current_dataset_idx[dataset_id]] )
    
        geod = Geodesic.WGS84.Inverse( start_coord[0], start_coord[1], current_point[0], current_point[1] )
        print(geod)
    
        if geod['s12'] < dist_thresh_m:
        
            #  Compute angle to new point
            temp_angles.append( geod['azi1'])
            print('Using Point from Dataset: {}. Dist: {}, Azimuth: {}'.format(dataset_id, geod['s12'], geod['azi1']))
            matching_points.append( (dataset_id, current_dataset_idx[dataset_id]) )
            current_dataset_idx[dataset_id] += 1
        else:
            pass
    avg_angle = mean_angle( temp_angles )
    print('avg-angle: {}'.format(avg_angle))

    #  Compute new point
    geod = Geodesic.WGS84.Direct( current_point[0],
                                  current_point[1],
                                  avg_angle,
                                  step_dist_m )
    current_point = ( geod['lat2'], geod['lon2'] )
    
    #  Update Current Indices
    for dataset_id in dataset_ids:
        
        # The next point will be the next point in the sequence which is "forward" of the point
        pass
    input('End of Loop')
    

{'lat1': 39.5989743, 'lon1': -104.8609468, 'lat2': 39.627165, 'lon2': -104.868584, 'a12': 0.028784683617945096, 's12': 3197.9074241114827, 'azi1': -11.83191571577791, 'azi2': -11.836785192616361}
{'lat1': 39.5989743, 'lon1': -104.8609468, 'lat2': 39.598961, 'lon2': -104.860858, 'a12': 6.992759921777846e-05, 's12': 7.768777666233938, 'azi1': 100.95719169116127, 'azi2': 100.95724829317874}
Using Point from Dataset: 1. Dist: 7.768777666233938, Azimuth: 100.95719169116127
{'lat1': 39.5989743, 'lon1': -104.8609468, 'lat2': 39.599353, 'lon2': -104.860711, 'a12': 0.0004200775682623222, 's12': 46.66954541145394, 'azi1': 25.719601426835418, 'azi2': 25.719751728760087}
avg-angle: 100.95719169116127


End of Loop 


{'lat1': 39.5989743, 'lon1': -104.8609468, 'lat2': 39.627165, 'lon2': -104.868584, 'a12': 0.028784683617945096, 's12': 3197.9074241114827, 'azi1': -11.83191571577791, 'azi2': -11.836785192616361}
{'lat1': 39.5989743, 'lon1': -104.8609468, 'lat2': 39.598936, 'lon2': -104.860909, 'a12': 4.815662345315682e-05, 's12': 5.350077861673171, 'azi1': 142.63794113256424, 'azi2': 142.6379652266599}
Using Point from Dataset: 1. Dist: 5.350077861673171, Azimuth: 142.63794113256424
{'lat1': 39.5989743, 'lon1': -104.8609468, 'lat2': 39.599353, 'lon2': -104.860711, 'a12': 0.0004200775682623222, 's12': 46.66954541145394, 'azi1': 25.719601426835418, 'azi2': 25.719751728760087}
avg-angle: 142.63794113256424


End of Loop 


{'lat1': 39.5989743, 'lon1': -104.8609468, 'lat2': 39.627165, 'lon2': -104.868584, 'a12': 0.028784683617945096, 's12': 3197.9074241114827, 'azi1': -11.83191571577791, 'azi2': -11.836785192616361}
{'lat1': 39.5989743, 'lon1': -104.8609468, 'lat2': 39.598921, 'lon2': -104.860941, 'a12': 5.3454579432126866e-05, 's12': 5.9386672360806125, 'azi1': 175.1880423448853, 'azi2': 175.18804604186238}
Using Point from Dataset: 1. Dist: 5.9386672360806125, Azimuth: 175.1880423448853
{'lat1': 39.5989743, 'lon1': -104.8609468, 'lat2': 39.599353, 'lon2': -104.860711, 'a12': 0.0004200775682623222, 's12': 46.66954541145394, 'azi1': 25.719601426835418, 'azi2': 25.719751728760087}
avg-angle: 175.1880423448853


...

End of Loop 


{'lat1': 39.5989743, 'lon1': -104.8609468, 'lat2': 39.627165, 'lon2': -104.868584, 'a12': 0.028784683617945096, 's12': 3197.9074241114827, 'azi1': -11.83191571577791, 'azi2': -11.836785192616361}
{'lat1': 39.5989743, 'lon1': -104.8609468, 'lat2': 39.598891, 'lon2': -104.860833, 'a12': 0.00012112264768595715, 's12': 13.45641676317516, 'azi1': 133.4164203585845, 'azi2': 133.41649289580107}
Using Point from Dataset: 1. Dist: 13.45641676317516, Azimuth: 133.4164203585845
{'lat1': 39.5989743, 'lon1': -104.8609468, 'lat2': 39.599353, 'lon2': -104.860711, 'a12': 0.0004200775682623222, 's12': 46.66954541145394, 'azi1': 25.719601426835418, 'azi2': 25.719751728760087}
avg-angle: 133.4164203585845


End of Loop 


{'lat1': 39.5989743, 'lon1': -104.8609468, 'lat2': 39.627165, 'lon2': -104.868584, 'a12': 0.028784683617945096, 's12': 3197.9074241114827, 'azi1': -11.83191571577791, 'azi2': -11.836785192616361}
{'lat1': 39.5989743, 'lon1': -104.8609468, 'lat2': 39.598853, 'lon2': -104.860834, 'a12': 0.00014933249147878642, 's12': 16.59045833142045, 'azi1': 144.26878258606, 'azi2': 144.26885448583812}
Using Point from Dataset: 1. Dist: 16.59045833142045, Azimuth: 144.26878258606
{'lat1': 39.5989743, 'lon1': -104.8609468, 'lat2': 39.599353, 'lon2': -104.860711, 'a12': 0.0004200775682623222, 's12': 46.66954541145394, 'azi1': 25.719601426835418, 'azi2': 25.719751728760087}
avg-angle: 144.26878258606


KeyboardInterrupt: Interrupted by user